In [29]:
import json
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense


def custom_scale(data):
    """Chuẩn hóa dữ liệu về khoảng 0.25 đến 1."""
    return data / 40000


def custom_inverse_scale(data):
    """Chuyển dữ liệu đã chuẩn hóa trở lại quy mô ban đầu."""
    return data * 40000


# Đọc dữ liệu từ tập tin JSON
file_path = "hsg.json"
with open(file_path, "r") as file:
    data = json.load(file)

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(
    {
        "Date": pd.to_datetime(data["t"], unit="s"),
        "Open": data["o"],
        "High": data["h"],
        "Low": data["l"],
        "Close": data["c"],
        "Volume": data["v"],
    }
)

# Sử dụng các giá trị hiện tại để dự đoán giá 'Open' của ngày tiếp theo
df["Next Open"] = df["Open"].shift(-1)
df.dropna(inplace=True)  # Loại bỏ bất kỳ hàng nào chứa NaN

# Chia dữ liệu thành features (X) và target (y)
X = df[["High", "Low", "Close", "Volume"]].values
y = df["Next Open"].values

# Chuẩn hóa dữ liệu
X_scaled = custom_scale(X)
y_scaled = custom_scale(y)

# Thêm một chiều vào X để phù hợp với đầu vào của GRU
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Xây dựng mô hình GRU
model = Sequential(
    [
        GRU(
            50,
            return_sequences=True,
            input_shape=(X_scaled.shape[1], X_scaled.shape[2]),
        ),
        GRU(50),
        Dense(1),
    ]
)

# Biên dịch mô hình
model.compile(optimizer="adam", loss="mean_squared_error")

# Huấn luyện mô hình trên toàn bộ dữ liệu
model.fit(X_scaled, y_scaled, epochs=128, batch_size=1024, verbose=1)

# Dự đoán giá mở cửa cho ngày tiếp theo
predicted_next_open_scaled = model.predict(
    X_scaled[-1].reshape(1, 1, X_scaled.shape[2])
)
predicted_next_open = custom_inverse_scale(predicted_next_open_scaled)

print(
    f"Giá mở cửa dự đoán cho ngày tiếp theo (đã đưa về quy mô ban đầu): {predicted_next_open[0][0]}"
)

Epoch 1/128
4/4 [==============================] - 3s 121ms/step - loss: 0.0322
Epoch 2/128
4/4 [==============================] - 0s 19ms/step - loss: 0.0261
Epoch 3/128
4/4 [==============================] - 0s 38ms/step - loss: 0.0241
Epoch 4/128
4/4 [==============================] - 0s 13ms/step - loss: 0.0208
Epoch 5/128
4/4 [==============================] - 0s 12ms/step - loss: 0.0215
Epoch 6/128
4/4 [==============================] - 0s 12ms/step - loss: 0.0198
Epoch 7/128
4/4 [==============================] - 0s 23ms/step - loss: 0.0199
Epoch 8/128
4/4 [==============================] - 0s 18ms/step - loss: 0.0196
Epoch 9/128
4/4 [==============================] - 0s 19ms/step - loss: 0.0190
Epoch 10/128
4/4 [==============================] - 0s 15ms/step - loss: 0.0189
Epoch 11/128
4/4 [==============================] - 0s 11ms/step - loss: 0.0187
Epoch 12/128
4/4 [==============================] - 0s 11ms/step - loss: 0.0182
Epoch 13/128
4/4 [==============================